# TUIA NLP 2024 TRABAJO PRÁCTICO 1

**Materia:** Procesamiento del Lenguaje Natural (IA 4.2)

**Año académico:** 2024

**Período lectivo:** 1° cuatrimestre

**Integrantes:**

- Britos Julián Francisco
- Moresco Brisa

#### Objetivo

El objetivo de este proyecto es desarrollar un programa que interactúe con el usuario para recomendar lecturas. 

#### Opciones principales

El programa ofrecerá tres opciones principales:

- Recomendación Directa: El programa preguntará al usuario "¿Qué tienes ganas de leer hoy?" y, mediante la clasificación de la respuesta, propondrá una lista de tres libros acordes a las temáticas mencionadas. Además, se detallará el autor, género y una breve reseña de cada libro que se recomienda descargar.

- Elección por Autor: Si el usuario prefiere buscar por autor, el programa ofrecerá una lista de libros del autor especificado. En caso de que haya múltiples resultados, se basará en la similitud de los dos primeros resultados más relevantes, retornando dos títulos (con sus respectivas reseñas) del resultado más cercano y uno del segundo que se recomiendan para descargar. Si existen varios libros del mismo autor aleatorizar. 

- Elección por Género Literario: Similar a la búsqueda por autor, si el usuario elige buscar por género, el programa ofrecerá una lista de libros del género especificado. Aplicará la misma lógica de similitud para seleccionar y presentar los resultados.


# Librerías

In [1]:
# !pip install streamlit
# !pip install requests
# !pip install beautifulsoup4

In [2]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
import pandas as pd
import os

# Obtención de url principal.

In [3]:
url_base = "https://ww3.lectulandia.com/"
response = requests.get(url_base)
soup = BeautifulSoup(response.text, 'html.parser')

# Encuentra el enlace relativo para la página de libros (/book/)
link_element = soup.select_one('#site-navigation > ul > li:nth-child(2) > a')
url_relativa = link_element['href']

# Combina la URL base con el URL relativo para obtener un URL absoluto (https://ww3.lectulandia.com/book/)
url_absoluta = urljoin(url_base, url_relativa)

# Hacemos una solicitud a la url absoluta para obtener la página a la que apunta el enlace, es decir la pagina con todos los libros
link_response = requests.get(url_absoluta)
link_soup = BeautifulSoup(link_response.text, 'html.parser')

# Crea la carpeta 'img' si aún no existe
if not os.path.exists('img'):
    os.makedirs('img')

# Obtención de datos

Primero, obtenemos cada página y cada libro en cada página. Luego obtenemos el título, género/s, autor/es, descripción y foto de cada libro. 

In [29]:
# Establecemos el número de páginas que queremos recorrer.
n_pags = 13 # No puede ser menor a 1, ya que la página principal cuenta como la primera página.

# Creamos una lista vacía para almacenar nuestros datos.
data = []

for i in range(13, n_pags + 1):
    # Hacemos una solicitud a la url de la página i.    
    link_response = requests.get(url_absoluta + f'page/{i}/')
    link_soup = BeautifulSoup(link_response.text, 'html.parser')
    
    
    # Iteramos sobre cada libro en la página.
    for libro in link_soup.select('article.card'):
        # Extrae el url de cada libro de la página i.
        url_relativa_libro = libro.select_one('h2 > a.title').get('href')
        # Combina la URL base con el URL relativo para obtener un URL absoluto (https://ww3.lectulandia.com/book/libro-de-ejemplo-1)
        url_absoluta_libro = urljoin(url_base, url_relativa_libro)
        # Imprimimos el url de cada libro.
        #print(f'url: {url_absoluta_libro}')
        # Creamos una lista con los urls de los libros y añadimos el url de cada libro a la lista.
        url_absoluta_libros = []
        url_absoluta_libros.append(url_absoluta_libro)
        

        # Hacemos una solicitud a la url de cada libro.
        libro_response = requests.get(url_absoluta_libro)
        libro_soup = BeautifulSoup(libro_response.text, 'html.parser')
            
        # Extraemos el título de cada libro.
        titulo = libro_soup.find(id='title').find('h1').text
        #print(f'Titulo: {titulo}')
        
        # Extraemos el autor de cada libro.
        autores_elements = libro_soup.find(id='autor').find_all('a')
        x_autores = [autor.text for autor in autores_elements]
        #print(f'Autores:\n   {x_autores}')
        # Creamos una lista con los autores de los libros y añadimos el autor de cada libro a la lista.
        autores = []
        autores.append(x_autores)
        
        # Extraemos el género de cada libro.
        generos_elements = libro_soup.find(id='genero').find_all('a')
        x_generos = [genero.text for genero in generos_elements]
        #print(f'Generos:\n   {x_generos}')
        # Creamos una lista con los géneros de los libros y añadimos el género de cada libro a la lista.
        generos = []
        generos.append(x_generos)
        
        # Extraemos la sinopsis de cada libro.
        sinopsis_element = libro_soup.find('div', class_='realign', id='sinopsis')
        if sinopsis_element is not None:
            span_element = sinopsis_element.find('span')
            if span_element is not None:
                sinopsis = span_element.text
            else:
                sinopsis = ''
        else:
            sinopsis = ''
        #print(sinopsis_element)
        #print(f'Sinopsis:\n   {sinopsis}')
        # Creamos una lista con las sinopsis de los libros y añadimos la sinopsis de cada libro a la lista.
        sinopsis_libros = []
        sinopsis_libros.append(sinopsis)
        
        # Descargamos la imagen de cada libro.
        img_url = libro_soup.find(id='cover').find('img').get('src')
        img_response = requests.get(img_url)
        img = img_response.content
        # Limpiamos el título para que no contenga caracteres no permitidos en un nombre de archivo.
        # Limpiamos el título para que no contenga caracteres no permitidos en un nombre de archivo.
        titulo_limpio = titulo.replace(' ', '_').replace(':', '').replace('/', '_').replace('?', '')
        # Guardamos la imagen en un archivo en la carpeta 'img'.
        with open(f'img/{titulo_limpio}.jpg', 'wb') as file:
            file.write(img)
        #print(f'Imagen guardada en img/{titulo_limpio}.jpg')
        
        data.append({
            'Titulo': titulo,
            'Autor': autores,
            'Genero': generos,
            'Descripcion': sinopsis
            })

        
libros = pd.DataFrame(data)

https://ww3.lectulandia.com/book/page/12/ ✔️


In [27]:
# Recorremos cada columna del DataFrame y, si el valor en esa columna es una lista con al menos un elemento, lo reemplazará con el primer elemento de la lista. Si el valor no es una lista o es una lista vacía, se mantendrá como está.
for column in libros.columns:
    libros[column] = libros[column].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x)

In [ ]:
libros.to_csv('libros.csv', index=False)

In [28]:
libros

,Titulo,Autor,Genero,Descripcion
0,Las niñas sin nombre,[Serena Burdick],"[Histórico, Novela]",Cerca de la mansión familiar de Luella y Effie...
1,El cuarto secreto,[Tono Galiana],"[Novela, Terror]","No podía ser de otra manera, tanto abuso, tant..."
2,La Balada de Nunca Jamás,[Stephanie Garber],"[Fantástico, Novela]","Los finales felices pueden atraparse, pero es ..."
3,Una herencia en juego,[Jennifer Lynn Barnes],"[Intriga, Juvenil, Novela]",Ella viene de la nada. Avery tiene un plan: pa...
4,La otra Gioconda,[Peio H. Riaño],"[Arte, Divulgación]","La Gioconda, el retrato más famoso del mundo, ..."
5,Dominando ChatGPT en 30 días,[Pablo Tapias Cantos],"[Ensayo, Informática, Tecnología]",¿Estás listo para dominar ChatGPT y destacarte...
6,Preparados para el amor,[Debbie Macomber],"[Novela, Romántico]","Con aquellas dos mujeres cerca, muy pronto los..."
7,Cuenta atrás,[Iris Johansen],"[Intriga, Novela, Policíaco]",La vida de Jane MacGuire parece cambiar para s...
8,El caso de la coqueta indiferente,[Jonathan Craig],"[Novela, Policíaco]",Eddie Macklin es empujado a la vía del metro y...
9,Los españoles pintados por si mismos,[Varios Autores],"[Realista, Relato]",Una relación de tipos o estampas costumbristas...
